In [191]:
from pynq import Overlay

ol = Overlay("./matrixmult.bit")

In [192]:
ol.ip_dict

{'transfer_kernel_0': {'fullpath': 'transfer_kernel_0',
  'type': 'xilinx.com:hls:transfer_kernel:1.0',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1073741824,
  'mem_id': 's_axi_CTRL',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_CTRL_ADDR_WIDTH': '7',
   'C_S_AXI_CTRL_DATA_WIDTH': '32',
   'C_M_AXI_MATA_ID_WIDTH': '1',
   'C_M_AXI_MATA_ADDR_WIDTH': '64',
   'C_M_AXI_MATA_DATA_WIDTH': '64',
   'C_M_AXI_MATA_AWUSER_WIDTH': '1',
   'C_M_AXI_MATA_ARUSER_WIDTH': '1',
   'C_M_AXI_MATA_WUSER_WIDTH': '1',
   'C_M_AXI_MATA_RUSER_WIDTH': '1',
   'C_M_AXI_MATA_BUSER_WIDTH': '1',
   'C_M_AXI_MATA_USER_VALUE': '0x00000000',
   'C_M_AXI_MATA_PROT_VALUE': '"000"',
   'C_M_AXI_MATA_CACHE_VALUE': '"0011"',
   'C_M_AXI_MATB_ID_WIDTH': '1',
   'C_M_AXI_MATB_ADDR_WIDTH': '64',
   'C_M_AXI_MATB_DATA_WIDTH': '64',
   'C_M_AXI_MATB_AWUSER_WIDTH': '1',
   'C_M_AXI_MATB_ARUSER_WIDTH': '1',
   'C_M_AXI_MATB_WUSER_WIDTH': '1',
   'C_M_AXI_MAT

In [193]:
ip = ol.transfer_kernel_0

In [194]:
from pynq import allocate
import numpy as np

np.set_printoptions(suppress=True)

mat1r = 5
mat1c_mat2r = 5
mat2c = 5

inbuff = allocate(shape=(mat1r*mat1c_mat2r,), dtype=np.uint64)
in2buff = allocate(shape=(mat1c_mat2r*mat2c,), dtype=np.uint64)
outbuff = allocate(shape=(mat1r*mat2c,), dtype=np.uint64)

In [195]:
ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0, CHAN2_INT_EN=0, RESERVED_1=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0, CHAN2_INT_ST=0, RESERVED_1=0),
  matA_offset_1 = Register(matA_offset=write-only),
  matA_offset_2 = Register(matA_offset=write-only),
  matB_offset_1 = Register(matB_offset=write-only),
  matB_offset_2 = Register(matB_offset=write-only),
  matout_offset_1 = Register(matout_offset=write-only),
  matout_offset_2 = Register(matout_offset=write-only),
  mat1r = Register(mat1r=write-only),
  mat1cmat2r = Register(mat1cmat2r=write-only),
  mat2c = Register(mat2c=write-only)
}

In [196]:
print("inbuff physical address {}".format(hex(inbuff.physical_address)))
print("in2buff physical address {}".format(hex(in2buff.physical_address)))
print("outbuff physical address {}".format(hex(outbuff.physical_address)))

inbuff physical address 0x1704f000
in2buff physical address 0x17066000
outbuff physical address 0x17067000


In [197]:
ip.register_map.matA_offset_1 = inbuff.physical_address
ip.register_map.matB_offset_1 = in2buff.physical_address
ip.register_map.matout_offset_1 = outbuff.physical_address
ip.register_map.mat1r = mat1r
ip.register_map.mat1cmat2r = mat1c_mat2r
ip.register_map.mat2c = mat2c

In [198]:

acc = 0
latch = 0

for i in range(mat1r*mat1c_mat2r):
    inbuff[i] = acc * 32768
    #in2buff[i] = acc * 32768
    if(latch == 0):
        acc+=0.0001
        if(acc > 1.000):
            latch = 1
            acc-=0.0001
    if(latch == 1):
        acc-=0.0001
        if(acc < 0):
            latch = 0
            acc+=0.0001

acc = 0
latch = 0

for i in range(mat1c_mat2r*mat2c):
    #inbuff[i] = acc * 32768
    in2buff[i] = acc * 32768
    if(latch == 0):
        acc+=0.0001
        if(acc > 1.000):
            latch = 1
            acc-=0.0001
    if(latch == 1):
        acc-=0.0001
        if(acc < 0):
            latch = 0
            acc+=0.0001

'''
acc = 0.0001

for i in range(65536):
    inbuff[i] = acc
    acc+=0.0001

acc = 0

for i in range(256):
    for j in range(i, 65536, 256):
        in2buff[j] = (j*0.0001)+0.0001
'''

np.savetxt('inbuff.csv', inbuff, delimiter=',', fmt='%f')
np.savetxt('in2buff.csv', in2buff, delimiter=',', fmt='%f')

In [199]:
outbuff

PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0], dtype=uint64)

In [200]:
#ip.register_map.CTRL.AP_START=1

In [201]:
import time

ip.register_map.CTRL.AP_START=1

start_time = time.time()

while ip.register_map.CTRL.AP_DONE == 0:
    time.sleep(0.0000001)

#print(ip.register_map.CTRL.AP_DONE)
print("--- %s seconds ---" % (time.time() - start_time))

# 100x100 * 100x100 -> 1 second
# 256x256 * 256x256 -> 3 seconds
# 1024x1024 * 1024x1024 -> 130 seconds ; 
# possible isuses: read request(), read(), write request(), write()

--- 0.0014071464538574219 seconds ---


In [202]:

#with np.printoptions(threshold=np.inf):
#    print(outbuff)
#outbuff

In [203]:
arrA = np.array(inbuff)
arr2dA = np.reshape(arrA, (mat1r,mat1c_mat2r))
np.savetxt('mm_matA.csv', arr2dA, delimiter=',', fmt='%f')

arrB = np.array(in2buff)
arr2dB = np.reshape(arrB, (mat2c,mat1c_mat2r))
np.savetxt('mm_matB.csv', arr2dB, delimiter=',', fmt='%f')

np.savetxt('outbuff.csv', outbuff, delimiter=',', fmt='%f')
outbuff_flt = np.divide(outbuff,32768**2)
arr = np.array(outbuff_flt)
arr2d = np.reshape(arr, (mat1r,mat2c))
np.savetxt('mm_result.csv', arr2d, delimiter=',', fmt='%f')

In [204]:
ip.register_map.CTRL.AUTO_RESTART=1

In [205]:
'''
import time
import numpy as np

acc = 0

inbuff=np.empty(1024*1024, dtype=float) 

for i in range(1048576):
    inbuff[i] = acc
    acc+=0.0001

acc = 0

in2buff=np.empty(1024*1024, dtype=float) 

for i in range(1024):
    for j in range(i, 1048576, 1024):
        in2buff[j] = (j*0.0001)+0.0001
        
arrA = np.array(inbuff)
arr2dA = np.reshape(arrA, (1024,1024))

arrB = np.array(in2buff)
arr2dB = np.reshape(arrB, (1024,1024))
'''
start_time = time.time()

outtie =np.empty(mat1r*mat2c, dtype=np.int64)
outtie = np.matmul(arr2dA,arr2dB.transpose())

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.001367330551147461 seconds ---


In [206]:
outtie = np.divide(outtie,32768**2).astype(float)

In [207]:
#np.array_equal(arr2d,outtie)  # test if same shape, same elements values
#np.array_equiv(arr2d,outtie)  # test if broadcastable shape, same elements values
#np.allclose(arr2d,outtie,...) # test if same shape, elements have close enough values

(arr2d==outtie).all()

True

In [208]:
np.savetxt('outtie.csv', outtie, delimiter=',', fmt='%f')

In [209]:
del inbuff, in2buff, outbuff